In [ ]:
from google.cloud import vision
from pdf2image import convert_from_path
from PIL import Image, ImageDraw, ImageFont
import io
import fitz  # PyMuPDF
import os
from xml.sax.saxutils import escape


In [ ]:
client = vision.ImageAnnotatorClient()

In [ ]:
def build_processed_pdf(directory, filename):
    annotated_images = []
    unannotated_images = []  # Unannotated images
    text_pages = []
    word_locations_list = []

    file_path = os.path.join(directory, filename)
    print(f"beginning to process: {file_path}")

    # Load PDF and extract image bytes from a given page
    doc = fitz.open(file_path)

    for page_num in range(doc.page_count):
        print(f"processing page: {page_num}...")
        
        page = doc.load_page(page_num)  # Load the first page
        images = page.get_images(full=True)
        xref = images[0][0]  # Get the first image reference
        base_image = doc.extract_image(xref)
        image_bytes = base_image["image"]  # Get the raw JPEG byte content

        # Convert to PIL Image for further processing if needed
        pil_image = Image.open(io.BytesIO(image_bytes))
        
        # Save unannotated image before drawing
        unannotated_images.append(pil_image.copy())

        # Annotate with vision
        image = vision.Image(content=image_bytes)
        response = client.text_detection(image=image)
        text_annotations = response.text_annotations

        # Build labeled image
        font_size = 15  
        font_path="/System/Library/Fonts/Supplemental/Arial.ttf"
        font = ImageFont.truetype(font_path, font_size)

        draw = ImageDraw.Draw(pil_image)
        for i, text_obj in enumerate(text_annotations):
            # Get the bounding box vertices
            vertices = [(vertex.x, vertex.y) for vertex in text_obj.bounding_poly.vertices]

            # Draw a polygon based on the bounding box
            if len(vertices) == 4:
                draw.polygon(vertices, outline="red", width=2)
                if i > 0: # first bounding box is whole text region
                    draw.text(vertices[0], text_obj.description, fill="red", font=font)

        full_page_text = text_annotations[0].description  # always the first text_annotation object
        word_locations = text_annotations[1:]
        word_locations_list.append(word_locations)
        text_pages.append(full_page_text)


    return text_pages, word_locations_list, annotated_images

In [ ]:
import os
import json

def save_annotated_text_and_image_data(root_dir, data_location, text_data, image_data):
    """
    Creates a directory structure to save images, text data, and an HTML gallery for viewing.
    
    Parameters:
        root_dir (str): The root directory to store data.
        data_location (str): Directory location under root_dir for storing files.
        text_data (list of str): List of extracted text for each page.
        image_data (list of tuples): List of tuples, where each tuple contains the image filename (str) 
                                     and the image data (as bytes or PIL image).
                                     
    """
    # Create directory paths
    base_dir = os.path.join(root_dir, data_location)
    os.makedirs(base_dir, exist_ok=True)

    images_dir = os.path.join(base_dir, "images")
    os.makedirs(images_dir, exist_ok=True)
    
    # File paths for JSONL and HTML files
    jsonl_file_path = os.path.join(base_dir, "extracted_text.jsonl")
    html_file_path = os.path.join(base_dir, "index.html")
    
    # Step 1: Save text data to JSONL
    with open(jsonl_file_path, "w", encoding="utf-8") as jsonl_file:
        for i, text in enumerate(text_data):
            entry = {
                "image": f"images/page{i+1}.jpg",  # Image filename
                "text": text
            }
            jsonl_file.write(json.dumps(entry) + "\n")
    
    # Step 2: Save images to the images directory
    for i, img in enumerate(image_data):
        image_path = os.path.join(images_dir, f"page{i+1}.jpg")
        img.save(image_path, format="JPEG")
    
    # Step 3: Generate HTML file for viewing
    with open(html_file_path, "w", encoding="utf-8") as html_file:
        html_file.write("<html><head><title>Annotated Image Gallery</title></head><body>\n")
        html_file.write("<h1>Annotated Image Gallery</h1>\n")
        
        for i, text in enumerate(text_data):
            img_filename = f"images/page{i+1}.jpg"
            html_file.write("<div class='image-entry'>\n")
            html_file.write(f"  <img src='{img_filename}' alt='Page {i+1}' style='width:100%; max-width:600px;'>\n")
            html_file.write(f"  <textarea readonly style='width:100%; height:150px;'>{text}</textarea>\n")
            html_file.write("</div><br>\n")
        
        html_file.write("</body></html>")

    print(f"Data successfully saved in {base_dir}")

# Example usage:
# save_annotated_text_and_image_data(root_dir="my_data", data_location="ocr_project", text_data=["text1", "text2"], image_data=[("img1.jpg", img_bytes), ("img2.jpg", img_bytes)])

In [ ]:
import os

def save_xml_text_file(root_dir, title_file, text_data):
    """
    Generates and saves an XML file with a title and pages, where each page contains text data.
    
    Parameters:
        root_dir (str): The root directory where the XML file will be saved.
        title_file (str): The title to be used in the XML file and filename.
        text_data (list of str): List of extracted text for each page.
    """
    # Define the XML file path
    xml_file_path = os.path.join(root_dir, title_file, f"full_OCR_text_{title_file}.xml")
    os.makedirs(os.path.dirname(xml_file_path), exist_ok=True)
    
    # Start writing the XML file
    with open(xml_file_path, "w", encoding="utf-8") as xml_file:
        # Write XML declaration and root element with title
        xml_file.write("<?xml version='1.0' encoding='UTF-8'?>\n")
        xml_file.write(f"<document>\n  <title>{escape(title_file)}</title>\n")
        
        # Add each page with its content, escaping the text for XML safety
        for i, text in enumerate(text_data):
            # Escape text content to handle special characters
            escaped_text = escape(text)
            # Write each page as <page page="1">...</page>
            xml_file.write(f"  <page page='{i+1}'>\n")
            xml_file.write(f"    {escaped_text}\n")
            xml_file.write("  </page>\n")
        
        # Close the root element
        xml_file.write("</document>")

    print(f"XML file successfully saved at {xml_file_path}")

In [ ]:
def full_pdf_process(pdf_dir, pdf_filename, output_dir):
    current_title, ext = os.path.splitext(pdf_filename)
    assert ext == ".pdf", f"Expected .pdf, got {ext}"
    text_pages, marked_images = build_processed_pdf(pdf_dir, pdf_filename)
    save_annotated_text_and_image_data(output_dir, current_title, text_pages, marked_images)
    save_xml_text_file(output_dir, current_title, text_pages)
    return text_pages, marked_images

In [ ]:
# directory to read pdfs from
pdf_dir = "../PDF/Phat_Giao_journals"

# directory to save output
output_dir = "../processed_journal_data"

In [ ]:
text, words, images = build_processed_pdf(pdf_dir, "phat-giao-viet-nam-1956-01.pdf")

In [ ]:
images[0].size

In [ ]:
words[0][0]

In [ ]:
type(words[0][0])

In [ ]:
text[3]

In [ ]:
def convert_page_words(word_info):
    
    converted_info = []
    for word in word_info:
        text = word.description
        x = [vertex.x for vertex in word.bounding_poly.vertices]
        y = [vertex.y for vertex in word.bounding_poly.vertices]
        assert len(x) == 4, "not four pointed bounding box."

        width_bottom = x[1] - x[0] # point order is: bottom-left, bottom-right, top-right, top-left
        width_top = x[2] - x[3]
        height_left = y[3] - y[0]
        height_right = y[2] - y[1] # this according to the layout of bounding poly

        if width_bottom != width_top:
            print(f"Warning bounding box width mismatch for: {text}, skipping.")
            continue
        if height_left != height_right:
            print(f"Warning bounding box height mismatch for: {text}, skipping.")
            continue
        
        new_info = [text, (x[0], y[0]), width_bottom, height_left]
        converted_info.append(new_info)
    
    return converted_info

In [ ]:
new_info = convert_page_words(words[3])

In [ ]:
new_info

In [ ]:
def collate_lines(words):
    """
    Collate OCR words into lines based on y-coordinates and line height, 
    assuming the words are already sorted by their y-coordinates.

    Args:
        words (list): List of words with their positions and dimensions 
                      in the form [text, (x, y), width, height].
    
    Returns:
        list: Collated lines in the form 
              [text_line, (start_x, start_y), (end_x, start_y), line_height].
    """
    if not words:
        return []
    
    # Sort words by y-coordinate (descending for top-to-bottom layout)
    # words = sorted(words, key=lambda w: (w[1][1], w[1][0]))
        
    collated_lines = []
    current_line = []
    current_y = words[0][1][1]
    current_height = words[0][3]

    for word in words:
        text, (x, y), width, height = word
        # Calculate tolerance for y-coordinate based on current line height
        tolerance = current_height * 0.75
        
        # Check if the word fits in the current line
        if abs(y - current_y) <= tolerance:
            current_line.append(word)
        else:
            # Finalize the current line
            if current_line:
                line_text = " ".join(w[0] for w in current_line)
                start_x = current_line[0][1][0]
                start_y = current_line[0][1][1]
                end_x = current_line[-1][1][0] + current_line[-1][2]
                collated_lines.append([line_text, (start_x, start_y), (end_x, start_y), current_height])
            
            # Start a new line
            current_line = [word]
            current_y = y
            current_height = height
    
    # Add the last line
    if current_line:
        line_text = " ".join(w[0] for w in current_line)
        start_x = current_line[0][1][0]
        start_y = current_line[0][1][1]
        end_x = current_line[-1][1][0] + current_line[-1][2]
        collated_lines.append([line_text, (start_x, start_y), (end_x, start_y), current_height])
    
    return collated_lines

In [ ]:
collate_lines(new_info)

In [ ]:
images[3]

In [ ]:
new_info

In [ ]:
text_pages, marked_images = full_pdf_process(pdf_dir, "phat-giao-viet-nam-1956-25-26.pdf", output_dir)

In [ ]:
all_files =  os.listdir(pdf_dir)

In [ ]:
all_files

In [ ]:
px_list = [all_files[2]] + all_files[6:16] + all_files[17:21]

In [ ]:
px_list

In [ ]:
for filename in px_list:
        #print(filename)  # Do something with each file path
        full_pdf_process(pdf_dir, filename, output_dir)